In [7]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('databases/merged_gtfs.db')
gtfs_names = ['routes', 'stop_times', 'stops', 'transfers', 'trips']

for name in gtfs_names:
    df = pd.read_csv(f"gtfs/{name}.txt")
    df.to_sql(name, conn, if_exists='replace', index=False)

# Close the connection
conn.close()

/tmp/ipykernel_37501/2957384310.py:9: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"gtfs/{name}.txt")


In [9]:
import sqlite3

def test_query(db_path: str, query: str):
    """
    Test function to run a SQL query on the specified SQLite database.

    :param db_path: Path to the SQLite database
    :param query: SQL query to execute
    :return: Results of the query
    """
    try:
        # Connect to the SQLite database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Execute the query
        cursor.execute(query)

        # Fetch all results
        results = cursor.fetchall()

        # Close the connection
        conn.close()

        return results
    except sqlite3.Error as e:
        return {"error": f"SQLite error: {str(e)}"}

# Path to the merged GTFS database
db_path = "merged_gtfs.db"

# Example query to fetch all routes from the 'routes' table
query = "SELECT * FROM routes LIMIT 5;"  # You can modify this query as needed

# Run the query
results = test_query(db_path, query)

# Print the results
if isinstance(results, list):
    for row in results:
        print(row)
else:
    print(results)  # In case of an error, print the error message


('5-T01-y08-1', 1, 'T1', None, 0, 'F47922', '000000')
('11-MGr-n-y08-1', 1, 'MGrn', None, 1, '0AAE53', '000000')
('11-MRe-1-y08-1', 1, 'MRed1', None, 1, 'E91D2F', '000000')
('11-MRe-2-y08-1', 1, 'MRed2', None, 1, 'E91D2F', '000000')
('10', 1, '10', 'Gold Souq Bus Stn - Al Quoz Bus Stn', 3, '6F2E90', 'FFFFFF')


In [1]:
import streamlit as st
from tools import GeocodingTool, GTFSCoordinatorTool, CurrentDateTime
from langchain.agents import initialize_agent, AgentType, Tool
from langchain_openai import ChatOpenAI

# Initialize tools
gtfs_tool = GTFSCoordinatorTool()
geocode_tool = GeocodingTool()
curr_dt_tool = CurrentDateTime()

# Initialize LLM with system message for proper query format
llm = ChatOpenAI(
                base_url="https://api.sambanova.ai/v1/",
                api_key=st.secrets["SAMBANOVA_API_KEY"], 
                streaming=True,
                model="Meta-Llama-3.1-70B-Instruct",
            )

# Define tools
tools = [
    Tool(
        name="GTFSCoordinatorTool",
        func=gtfs_tool.run,
        description="Handles queries related to public transit data such as schedules, routes, and stop information"
    ),
    Tool(
        name="GeocodingTool",
        func=geocode_tool.run,
        description="Fetches geocoding information from TomTom's API based on user inputs"
    ),
    Tool(
        name="CurrentDateTime",
        func=curr_dt_tool.run,
        description="Returns the current date and time."
    )
]

sys_msg = """
You are an intelligent assistant specializing in public transit for Dubai. Your role is to assist users with queries related to transit schedules, routes, stops, and geolocation. Follow these instructions:

Understand the User Query:
Analyze the user's input to determine whether it requires:

Geolocation data (via GeocodingTool).
Transit schedule or stop information (via GTFSCoordinatorTool).
Time-sensitive data (via CurrentDateTime).
Provide Accurate Responses:

Use the GTFSCoordinatorTool to query transit data from the GTFS database.
Use the GeocodingTool to resolve place names into precise geographic coordinates.
Use CurrentDateTime to determine or adjust for time-based responses.
Output Format:
Ensure responses are clear and structured:

Provide transit details like schedules, routes, or stops in an easy-to-read format.
Return geolocation results with relevant coordinates and descriptions.
Include time-based context (current or user-specified) when appropriate.

Stay Contextual:

Respond in a concise, Dubai-specific context.
Include relevant details about transit services like bus, metro, and tram lines in Dubai.
Collaborate with Tools:
Use the tools efficiently to deliver accurate and timely information. Avoid making assumptions about transit data or geolocation without querying the relevant tool.
"""

# Initialize agent with tools and system message
agent = initialize_agent(
    tools, llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, system_message=sys_msg
)

# Test query
query = "When is the next E101 bus leaving from Ibn Battuta MS"
response = agent.run(query)
print("Response: ", response)


/tmp/ipykernel_70233/1218285563.py:68: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
/tmp/ipykernel_70233/1218285563.py:74: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(query)




> Entering new AgentExecutor chain...
To find out when the next E101 bus is leaving from Ibn Battuta MS, I need to access public transit data, specifically schedules and stop information. 

Action: GTFSCoordinatorTool
Action Input: {"stop": "Ibn Battuta MS", "route": "E101"}SELECT T2.arrival_time, T2.departure_time 
FROM stops AS T1 
INNER JOIN stop_times AS T2 
ON T1.stop_id = T2.stop_id 
INNER JOIN trips AS T3 
ON T2.trip_id = T3.trip_id 
INNER JOIN routes AS T4 
ON T3.route_id = T4.route_id 
WHERE T1.stop_name = 'Ibn Battuta MS' AND T4.route_short_name = 'E101' 
LIMIT 5

Observation: {'query': "SELECT T2.arrival_time, T2.departure_time \nFROM stops AS T1 \nINNER JOIN stop_times AS T2 \nON T1.stop_id = T2.stop_id \nINNER JOIN trips AS T3 \nON T2.trip_id = T3.trip_id \nINNER JOIN routes AS T4 \nON T3.route_id = T4.route_id \nWHERE T1.stop_name = 'Ibn Battuta MS' AND T4.route_short_name = 'E101' \nLIMIT 5", 'results': [], 'errors': None}
Thought:It seems like the GTFSCoordinatorTool 

In [6]:
import requests

tl_key = st.secrets["TRANSITLAND_API_KEY"]
response = requests.get(f"https://transit.land/api/v2/routing/otp/plan?fromPlace=37.7757,-122.47996&toPlace=37.76845,-122.23508&date=2024-05-03&time=17%3A59&api_key={tl_key}")

In [7]:
response

<Response [401]>